#### Pydantic

In [ ]:
!pip install pydantic==2.10.6
!pip install pydantic[Email]

In [43]:
# Importing required library
import pydantic
from typing import List, Optional, Any
from pydantic import BaseModel,EmailStr,PositiveInt,conlist,Field,HttpUrl,field_validator,model_validator
print(pydantic.__version__)

2.10.6


##### Data Validation without Pydantic & with Pydantic

In [4]:
# Data Validation without pydantic
class User:
    def __init__(self,id:int,name="Hrishikesh Kothawade"):
        if not isinstance(id,int):
            raise TypeError(f"Expected id to be an int, got {type(id).__name__}")
        if not isinstance(name,str):
            raise TypeError(f"Expected name to be an str, got {type(name).__name__}")
        
        self.id = id
        self.name = name
        
try:
    user = User(id="123")
except TypeError as te:
    print(te)

Expected id to be an int, got str


In [ ]:
# Data Validation with pydantic
class User(BaseModel):
    id:int
    name:str="Hrishikesh Kothawade"
    
# If I put string as a id then, pydantic will try to convert it into integer, if not possible then raises an error
user = User(id='123')
print(user.id)

# Unable to parse the 
user2 = User(id='Hrishikesh Kothawade')
print(user2.id)

In [13]:
# printing the model_field_set
print(user.model_fields_set)
user=User(id='123',name="Hrishikesh")
print(user.model_fields_set)

{'id'}
{'id', 'name'}


In [15]:
# dump the arguments into structure format
# Generate a dictionary representation of the model, optionally specifying which fields to include or exclude.
print(user.model_dump())

# Convert into json format
print(user.model_dump_json())

# Convert into json schema format
print(user.model_json_schema())

{'id': 123, 'name': 'Hrishikesh'}
{"id":123,"name":"Hrishikesh"}
{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'default': 'Hrishikesh Kothawade', 'title': 'Name', 'type': 'string'}}, 'required': ['id'], 'title': 'User', 'type': 'object'}


##### Nessted models

In [19]:
class Food(BaseModel):
    name:str
    price:float
    ingredients:Optional[List[str]] = None
    
class Restaurant(BaseModel):
    name:str
    location:str
    foods: List[Food]
    
    
restaurant_instance =  Restaurant(
    name="Tasty Burger",
    location="123, high street",
    foods=[
        {"name":"Cheese Pizza",'price':12.50,"ingredients":["Cheese","Tomato Sauce","Dough"]},
        {"name":"Veggie Burger","price":8.99}
    ]
)

# instance information
print(restaurant_instance)

# Information in specific format
print(restaurant_instance.model_dump())

name='Tasty Burger' location='123, high street' foods=[Food(name='Cheese Pizza', price=12.5, ingredients=['Cheese', 'Tomato Sauce', 'Dough']), Food(name='Veggie Burger', price=8.99, ingredients=None)]
{'name': 'Tasty Burger', 'location': '123, high street', 'foods': [{'name': 'Cheese Pizza', 'price': 12.5, 'ingredients': ['Cheese', 'Tomato Sauce', 'Dough']}, {'name': 'Veggie Burger', 'price': 8.99, 'ingredients': None}]}


##### Additional Parsers

In [22]:
class Address(BaseModel):
    street:str
    city:str
    zip_code:str
    state:str
    
class Employee(BaseModel):
    name:str
    position:str
    email:EmailStr
    
class Owner(BaseModel):
    name:str
    email:EmailStr
    
class Restaurant(BaseModel):
    # ... -> tells us that its required property
    name:str = Field(..., pattern=r"^[a-zA-Z0-9-' ]+$")
    owner:Owner
    address:Address
    employees:conlist(Employee,min_length=2)
    number_of_seats:PositiveInt
    delivery: bool
    website: HttpUrl


In [26]:
# Create instance of restaurant class
restaurant_instance = Restaurant(
    name="Tasty Bites",
    owner={
        "name":"Hrishikesh Kothawade",
        "email":"hrishikesh.kothawade@hrtech.com"
    },
    address={
        "street":"123, Flavour Street",
        "city":"Tastytown",
        "state":"MH",
        "zip_code":'424001'
    },
    employees=[
        {
        "name":"hrishikesh",
        "position":"CEO",
        "email":"hrishikesh.kothawade@hrtech.com"
         },
        {
        "name":"sachin",
        "position":"CFO",
        "email":"sachin.kukriti@hrtech.com"
         },
        {
        "name":"Banchoddas",
        "position":"COO",
        "email":"banshoddas.chanchad@hrtech.com"
         }
    ],
    number_of_seats=120,
    delivery=True,
    website="https://hrtech.com"
)
print(restaurant_instance)


name='Tasty Bites' owner=Owner(name='Hrishikesh Kothawade', email='hrishikesh.kothawade@hrtech.com') address=Address(street='123, Flavour Street', city='Tastytown', zip_code='424001', state='MH') employees=[Employee(name='hrishikesh', position='CEO', email='hrishikesh.kothawade@hrtech.com'), Employee(name='sachin', position='CFO', email='sachin.kukriti@hrtech.com'), Employee(name='Banchoddas', position='COO', email='banshoddas.chanchad@hrtech.com')] number_of_seats=120 delivery=True website=HttpUrl('https://hrtech.com/')


##### Field Validators

In [39]:
class Owner(BaseModel):
    name:str
    email:EmailStr
    
    # We mention 'name' in field_validator
    @field_validator('name')
    @classmethod
    def name_must_contain_space(cls,v:str) -> str:
        if ' ' not in v:
            raise ValueError("Owner name must contains space")
        return v.title()
    
try:
    owner_instance = Owner(name="hrishikesh Kothawade", email="hrishikesh.kothawade@hrtech.com")
except ValueError as e:
    print(e)
# Owner(name="HrishikeshKothawade",email="hrishikesh.kothawade@hrtech.com")

In [41]:
print(owner_instance)
# .title() makes it like title
owner_instance.name

name='Hrishikesh Kothawade' email='hrishikesh.kothawade@hrtech.com'


'Hrishikesh Kothawade'

##### Model Validators

In [49]:
class Owner(BaseModel):
    name:str
    email:EmailStr
    
    @model_validator(mode="before")
    @classmethod
    def check_sensitive_info_omitted(cls,data:Any) -> Any:
        if isinstance(data,dict):
            if 'password' in data:
                raise ValueError('password should not be included')
            if 'card_number' in data:
                raise ValueError("card_number should not be included")
        return data
    
    @model_validator(mode="after")  
    def check_name_contains_space(self) -> Owner:
        if ' ' not in self.name:
            raise ValueError("Owner name must contains space")
        return self
    
print(Owner(name="Hrishikesh Kothawade",email="hrishikesh.kothawade@hrtech.com"))

   
    

name='Hrishikesh Kothawade' email='hrishikesh.kothawade@hrtech.com'


In [50]:
try:
    Owner(name=123,email="hrishikesh.kothawade@hrtech.com",password="password123")
except Exception as e:
    print(e) 

1 validation error for Owner
  Value error, password should not be included [type=value_error, input_value={'name': 123, 'email': 'h...assword': 'password123'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error
